In [1]:
import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d
import matplotlib as mpl
from matplotlib.colors import LogNorm
from IPython.display import display, Markdown
from collections import OrderedDict
import pylhe
import glob
import pyslha

delphesDir = os.path.abspath("../MG5/Delphes")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT
import xml.etree.ElementTree as ET


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

Welcome to JupyROOT 6.26/04


# Set output and input files

In [2]:
# inputFile = '../DMSimp_axial_all/Events/run_04/axial_delphes_events.root'
# inputFile = '../DMSimp_axial_0j/Events/run_01/axial_delphes_events.root'
inputFile = '../DMSimp_axial_1j/Events/run_01/axial_delphes_events.root'

outputFile = inputFile.replace('delphes_events.root','cms_exo_20_004.pcl')

### Define dictionary to store data

In [3]:
dataDict = {'filename' : os.path.abspath(inputFile)}
cmsColumns = ['Coupling', 'Mode', '$m_{med}$', '$m_{DM}$', '$g_{\chi}$', '$g_{q}$',
       'Data-takingperiod', 'Fullsample', 'Triggeremulation',
       '$p_{T}^{miss}>250$GeV', '$p_{T}^{miss}$qualityfilters', 'Electronveto',
       'Muonveto', 'Tauveto', 'Bjetveto', 'Photonveto',
       '"$\Delta\phi(jet_p_{T}^{miss})>0.5$rad"',
       '$\Deltap_{T}^{miss}$(PF-Calorimeter)$<0.5$rad',
       'LeadingAK4jet$p_{T}>100$GeV', 'LeadingAK4jet$\eta<2.4$',
       'LeadingAK4jetenergyfractions', 'Mono-Voverlapremoval',
       'HCALmitigation(jets)', 'HCALmitigation($\phi^{miss}$)',
       '"$\Delta\phi(\mathrm{PF}_\mathrm{Charged})<2.0$rad"']
for column in cmsColumns:
    dataDict[column] = None

## Get Model Parameters

In [4]:
banner = sorted(glob.glob(os.path.dirname(inputFile)+'/*banner.txt'),key=os.path.getmtime,reverse=True)
if len(banner) == 0:
    print('Banner not found for %s' %label)
elif len(banner) > 1:        
    print('\n%i banner files found for %s. Using %s' 
          %(len(banner),label,os.path.basename(banner[0])))
banner = banner[0]
xtree = ET.parse(banner)
xroot = xtree.getroot()
#     xsecPB = eval(xroot.find('init').text.split()[-2])
slha = xroot.find('header').find('slha').text
pars = pyslha.readSLHA(slha)
mMed = pars.blocks['MASS'][55]
mDM = pars.blocks['MASS'][52]
gVq = pars.blocks['DMINPUTS'][4] # Mediator-quark vector coupling
gAq = pars.blocks['DMINPUTS'][10] # Mediator-quark axial coupling
gVx = pars.blocks['DMINPUTS'][2] # Mediator-DM vector coupling
gAx = pars.blocks['DMINPUTS'][3] # Mediator-DM axial coupling

print('mMed = %1.2f GeV, mDM = %1.2f GeV, gVq = %1.2f, gAq = %1.2f, gVx = %1.2f, gAx = %1.2f' 
      %(mMed,mDM,gVq,gAq,gVx,gAx))

mMed = 2000.00 GeV, mDM = 1.00 GeV, gVq = 0.00, gAq = 0.25, gVx = 0.00, gAx = 1.00


#### Store data

In [5]:
if gVx != 0:
    dataDict['Coupling'] = 'Vector'
else:
    dataDict['Coupling'] = 'Axial'
    
dataDict['Mode'] = '$\\chi\\chi+j$'

dataDict['$m_{med}$'] = mMed
dataDict['$m_{DM}$'] = mDM
if dataDict['Coupling'] == 'Vector':
    dataDict['$g_{\chi}$'] = gVx
    dataDict['$g_{q}$'] = gVq
else:
    dataDict['$g_{\chi}$'] = gAx
    dataDict['$g_{q}$'] = gAq

dataDict['Data-takingperiod'] = 2017

# Load events, apply cuts and store relevant info

## Cuts

In [6]:
## Trigger efficiency
triggerEff = 0.9 # Applied to the event weights

## jets
pTj1min = 100.
pTjmin = 25.
etamax = 2.4
## MET
minMET = 250.
## Electrons
pTmin_el = 10.
etamax_el = 2.5
nMax_el = 0
## Photons
pTmin_a = 15.
etamax_a = 2.5
nMax_a = 0
## Muons
pTmin_mu = 10.
etamax_mu = 2.4
nMax_mu = 0
## Tau jets
nMax_tau = 0
## b jets
nMax_b = 0

In [7]:
pTj1 = np.array([])
weights = np.array([])
met = np.array([])
dmPT = np.array([])
njets = np.array([])
totalweight = 0.0
cutFlow = { 'Fullsample' : 0.,
            'Triggeremulation' : 0.,
           '$p_{T}^{miss}>250$GeV' : 0., 
           'Electronveto' : 0.,
           'Muonveto' : 0., 
           'Tauveto' : 0., 
           'Bjetveto' : 0., 
           'Photonveto' : 0.,
           '"$\Delta\phi(jet_p_{T}^{miss})>0.5$rad"' : 0.,
           'LeadingAK4jet$p_{T}>100$GeV' : 0., 
           'LeadingAK4jet$\eta<2.4$' : 0.}


f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()
dataDict['Total MC Events'] = nevts

for ievt in range(nevts):    
    
    tree.GetEntry(ievt)        

    jets = tree.Jet
    weight = tree.Weight.At(1).Weight
    totalweight += weight

    missingET = tree.MissingET.At(0)
#         missingET = tree.GenMissingET.At(0)  # USE REAL MISSING ET!
    electrons = tree.Electron
    muons = tree.Muon
    photons = tree.Photon

    # Filter electrons:
    electronList = []
    for iel in range(electrons.GetEntries()):
        electron = electrons.At(iel)
        if electron.PT < pTmin_el:
            continue
        if abs(electron.Eta) > etamax_el:
            continue
        electronList.append(electron)

    # Filter muons:
    muonList = []
    for imu in range(muons.GetEntries()):
        muon = muons.At(imu)
        if muon.PT < pTmin_mu:
            continue
        if abs(muon.Eta) > etamax_mu:
            continue
        muonList.append(muon)

    # Filter photons:
    photonList = []
    for ia in range(photons.GetEntries()):
        photon = photons.At(ia)
        if photon.PT < pTmin_a:
            continue
        if abs(photon.Eta) > etamax_a:
            continue
        photonList.append(photon)            

    # Filter jets
    jetList = []
    bjetList = []
    taujetList = []
    for ijet in range(jets.GetEntries()):
        jet = jets.At(ijet)
        if jet.PT < pTjmin:
            continue
        if abs(jet.Eta) > 2.5:
            continue
        if jet.BTag:
            bjetList.append(jet)
        elif jet.TauTag:
            taujetList.append(jet)
        else:
            jetList.append(jet)  
    jetList = sorted(jetList, key = lambda j: j.PT, reverse=True)    

    if len(jetList) > 0:
        deltaPhi = np.abs(jetList[0].Phi-missingET.Phi) 
    else:
        deltaPhi = 0.0
        
    cutFlow['Fullsample'] += 1./nevts

    # Apply cuts:
    ## Apply trigger efficiency
    if np.random.uniform() > triggerEff:
        continue
    cutFlow['Triggeremulation'] += 1./nevts
    ## Cut on MET
    if missingET.MET < minMET: continue              
    cutFlow['$p_{T}^{miss}>250$GeV'] += weight        
    ## Veto electrons
    if len(electronList) > nMax_el: continue  
    cutFlow['Electronveto'] += 1./nevts
    ## Veto muons
    if len(muonList) > nMax_mu: continue  
    cutFlow['Muonveto'] += 1./nevts
    ## Veto tau jets
    if len(taujetList) > nMax_tau: continue  
    cutFlow['Tauveto'] += 1./nevts
    ## Veto b jets
    if len(bjetList) > nMax_b: continue  
    cutFlow['Bjetveto'] += 1./nevts
    ## Veto photons
    if len(photonList) > nMax_a: continue  
    cutFlow['Photonveto'] += 1./nevts           
    ## Delta Phi cut
    if deltaPhi < 0.5: continue
    cutFlow['"$\Delta\phi(jet_p_{T}^{miss})>0.5$rad"'] += 1./nevts               
    ## Jet cuts
    if len(jetList) < 1 or jetList[0].PT < pTj1min: continue
    cutFlow['LeadingAK4jet$p_{T}>100$GeV'] += 1./nevts
    if abs(jetList[0].Eta) > etamax: continue
    cutFlow['LeadingAK4jet$\eta<2.4$'] += 1./nevts  

    # Store relevant data        
    weights = np.append(weights,weight)
    met = np.append(met,missingET.MET)

f.Close()


### Store cut-flow

In [8]:
dataDict.update(cutFlow)

### Get global info and weight in MET bins

In [9]:
dataDict['Total xsec (pb)'] = totalweight
lumi2017 = 41.5
dataDict['Luminosity (1/fb)'] = lumi2017

metBins = [250,  280,  310,  340,  370,  400,  430,  470,  510, 550,  590,  640,  690,  
           740,  790,  840,  900,  960, 1020, 1090, 1160, 1250, 99999]
binc,binEdges = np.histogram(met,bins=metBins, weights=weights)
for ibin,b in enumerate(binc):
    label = 'bin_%1.1f_%1.1f'%(binEdges[ibin],binEdges[ibin+1])
    dataDict[label] = b*lumi2017


In [10]:
# Convert dataDict to a list with a single entry
for key,val in dataDict.items():
    dataDict[key] = [val]

#### Create pandas DataFrame

In [11]:
df = pd.DataFrame.from_dict(dataDict)

In [12]:
df

,filename,Coupling,Mode,$m_{med}$,$m_{DM}$,$g_{\chi}$,$g_{q}$,Data-takingperiod,Fullsample,Triggeremulation,...,bin_690.0_740.0,bin_740.0_790.0,bin_790.0_840.0,bin_840.0_900.0,bin_900.0_960.0,bin_960.0_1020.0,bin_1020.0_1090.0,bin_1090.0_1160.0,bin_1160.0_1250.0,bin_1250.0_99999.0
0,/home/lessa/MonoXSMS/DMSimp_axial_1j/Events/ru...,Axial,$\chi\chi+j$,2000.0,1.0,1.0,0.25,2017,1.0,0.90016,...,0.024708,0.022777,0.017759,0.012354,0.01081,0.008493,0.006563,0.003861,0.005019,0.01081


### Save DataFrame to pickle file

In [13]:
print('Saving to',outputFile)
df.to_pickle(outputFile)

Saving to ../DMSimp_axial_1j/Events/run_01/axial_cms_exo_20_004.pcl
